Copyright (c) 2019 [윤기태]

https://github.com/yoonkt200/python-data-analysis

[MIT License](https://github.com/yoonkt200/python-data-analysis/blob/master/LICENSE.txt)

# (가제) 파이썬 데이터 분석

-----

# 웹 페이지 크롤링

### 1. 대상 페이지의 구조 살펴보기

- Internet Explorer, Chrome 등의 웹 브라우저로 크롤링의 대상이 될 페이지 구조 살펴보기
    - 웹 브라우저의 개발자 도구 실행 (Chrome 기준 설명)
        - `Chrome 맞춤설정 및 제어 탭 - 도구 더보기 - 개발자 도구` 클릭
        - 혹은 `Command + Alt + I (Mac), Ctrl + Shift + I (윈도우)`

![앱 생성 페이지](img/crawl_1.png)

-----

### 2. 페이지들의 URL 정보 추출

- 개발자 도구를 통해, 텍스트 데이터를 가져올 페이지들의 URL 리스트 추출
    - HTML 구조에서 링크 정보 (**a 태그**) 추출
        - 개발자도구 좌측 상단의 `마우스 모양` 버튼 클릭
        - HTML 구조를 보고싶은 영역으로 마우스 커서 이동

![앱 생성 페이지](img/crawl_2.png)

In [1]:
import requests
from bs4 import BeautifulSoup
import re

source_url = "https://namu.wiki/RecentChanges"

req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="table", attrs={"class":"table-hover"})
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        page_urls.append(page_url)

# 중복 url 제거
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

https://namu.wiki/w/%ED%8C%8C%EC%9D%BC:gh_range_2line.png
https://namu.wiki/w/Rise%20of%20the%20Blood%20Elves
https://namu.wiki/w/%EB%B8%94%EB%9E%99%20%EB%AF%B8%EB%9F%AC
https://namu.wiki/w/%EC%9C%A0%EC%83%81%ED%9B%88
https://namu.wiki/w/%ED%95%98%EC%8A%A4%EC%8A%A4%ED%86%A4/%EC%B9%B4%EB%93%9C%EC%9D%BC%EB%9E%8C/%EC%96%B4%EB%91%A0%EC%9D%98%20%EB%B0%98%EA%B2%A9/%EC%9D%BC%EB%B0%98%20%EB%93%B1%EA%B8%89


- 위의 코드를 통해 https://namu.wiki/RecentChanges 페이지의 최근 변경 문서 링크들을 추출.

-----

### 3. 각종 텍스트 정보 추출

- 동일한 방법으로, 개발자 도구를 이용하여 최근 변경된 나무위키 페이지의 텍스트 정보 추출
    - `문서 제목`
    - `문서 카테고리`
    - `문서 내용`

![앱 생성 페이지](img/crawl_3.png)

![앱 생성 페이지](img/crawl_4.png)

![앱 생성 페이지](img/crawl_5.png)

- `2. 페이지들의 URL 정보 추출` 에서 추출했던 URL 페이지 정보를 기반으로 크롤링 실행.

In [2]:
req = requests.get(page_urls[0])
html = req.content
soup = BeautifulSoup(html, 'lxml')
title = soup.find(name="h1", attrs={"class":"title"})
category = soup.find(name="div", attrs={"class":"wiki-category"})
content_clearfix = soup.find(name="div", attrs={"class":"wiki-content clearfix"})
print(title.text)
print(category.text)
print(content_clearfix.text)


파일:gh_range_2line.png


분류

파일/보드게임/아그리콜라





이 파일은 나무위키에서 제한된 한도 안에서 쓰입니다.본 이미지는 퍼블릭 도메인 혹은 자유이용 저작물이 아닌, 독점적 저작권이 존재하는 이미지입니다. 나무위키는 본 이미지를 제한된 한도 안에서 이용합니다.※ 다음과 같은 사항을 참조하시기 바랍니다.권리자는 해당 이미지가 나무위키에 게시되어 권리자의 정당한 권리를 침해한다고 여겨진다면 문의 게시판에 문의해 주세요.해당 이미지를 업로드한 사용자는 자신의 행위에 대한 법적 책임이 부과될 수 있음을 숙지해주세요.1. 기본 정보2. 이미지 설명1. 기본 정보[편집]출처출처를 삽입해 주세요.날짜이미지가 만들어진 날짜를 삽입해 주세요.저작자이미지의 저작자를 삽입해 주세요.저작권이미지의 저작권과 관련된 기타 정보를 삽입해 주세요.기타기타 정보가 있으면 삽입해 주세요.2. 이미지 설명[편집]이미지의 자세한 설명을 적어 주세요.


